In [1]:
import json
import os

DATA_FILE = "biomarket_data.json"

def load_data():
    """
    Load data from the JSON file. If the file does not exist, create an empty data structure.
    Returns a dictionary with 'products' and 'sales'.
    """
    if not os.path.exists(DATA_FILE):
        return {"products": {}, "sales": []}
    try:
        with open(DATA_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    except (json.JSONDecodeError, IOError):
        print("Error loading data. File corrupted or unreadable.")
        return {"products": {}, "sales": []}


In [2]:
def save_data(data):
    """
    Save data to the JSON file for persistence.
    """
    with open(DATA_FILE, "w") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

In [3]:
def print_help():
    """
    Print the help menu with available commands.
    """
    print("Available commands are:")
    print("add: add a product to the inventory")
    print("list: list all products in inventory")
    print("sale: register a completed sale")
    print("profits: show total profits")
    print("help: show available commands")
    print("exit: exit the program")

In [4]:
def input_positive_int(prompt):
    """
    Prompt the user for a positive integer input, repeat until valid.
    """
    while True:
        val = input(prompt).strip()
        if val.isdigit() and int(val) > 0:
            return int(val)
        print("Invalid value. Please enter a positive integer.")

In [5]:
def input_positive_float(prompt):
    """
    Prompt the user for a positive float input, repeat until valid.
    """
    while True:
        val = input(prompt).strip().replace(",", ".")
        try:
            f = float(val)
            if f > 0:
                return f
            else:
                print("Please enter a positive number.")
        except ValueError:
            print("Invalid value. Please enter a number (e.g. 1.40).")


In [6]:
def add_product(data):
    """
    Add a product to the inventory. If the product already exists,
    only update the quantity.
    """
    name = input("Product name: ").strip().lower()
    if not name:
        print("Invalid product name.")
        return
    quantity = input_positive_int("Quantity: ")

    products = data["products"]
    if name in products:
        # Existing product: add quantity
        products[name]["quantity"] += quantity
        print(f"ADDED: {quantity} X {name}")
    else:
        # New product: ask for prices
        price_purchase = input_positive_float("Purchase price: ")
        price_sale = input_positive_float("Sale price: ")
        products[name] = {
            "quantity": quantity,
            "price_purchase": price_purchase,
            "price_sale": price_sale
        }
        print(f"ADDED: {quantity} X {name}")


In [7]:
def list_products(data):
    """
    List all products currently in inventory.
    """
    products = data["products"]
    if not products:
        print("Inventory is empty.")
        return
    print("PRODUCT        QUANTITY   PRICE")
    for name, info in products.items():
        print(f"{name:<15} {info['quantity']:<10} €{info['price_sale']}")

In [8]:
def register_sale(data):
    """
    Register a sale, updating inventory and saving the sale.
    Check product availability and quantity.
    """
    products = data["products"]
    sale_items = []
    while True:
        name = input("Product name: ").strip().lower()
        if name not in products:
            print(f"Error: product '{name}' not found in inventory.")
            continue
        max_qty = products[name]["quantity"]
        if max_qty == 0:
            print(f"Error: product '{name}' is out of stock.")
            continue
        quantity = input_positive_int("Quantity: ")
        if quantity > max_qty:
            print(f"Error: available quantity for '{name}' is {max_qty}.")
            continue
        sale_items.append({"name": name, "quantity": quantity})
        another = input("Add another product? (yes/no): ").strip().lower()
        while another not in ("yes", "no"):
            another = input("Invalid answer. Add another product? (yes/no): ").strip().lower()
        if another == "no":
            break

    # Update inventory and record sale
    total = 0
    print("SALE REGISTERED")
    for item in sale_items:
        p = products[item["name"]]
        p["quantity"] -= item["quantity"]
        line_total = item["quantity"] * p["price_sale"]
        total += line_total
        print(f"- {item['quantity']} X {item['name']}: €{p['price_sale']}")

    print(f"Total: €{round(total, 2)}")

    # Save the sale for profit calculations
    data["sales"].append(sale_items)


In [9]:
def calculate_profits(data):
    """
    Calculate and show gross and net profits.
    Gross profit = total sales revenue (sale price * quantity)
    Net profit = gross profit - purchase cost of sold products
    """
    products = data["products"]
    sales = data["sales"]
    total_revenue = 0.0
    total_cost = 0.0

    for sale in sales:
        for item in sale:
            name = item["name"]
            qty = item["quantity"]
            if name in products:
                price_sale = products[name]["price_sale"]
                price_purchase = products[name]["price_purchase"]
            else:
                # If product no longer in inventory, skip for simplicity
                continue
            total_revenue += price_sale * qty
            total_cost += price_purchase * qty

    gross_profit = round(total_revenue, 2)
    net_profit = round(total_revenue - total_cost, 2)
    print(f"Profit: gross=€{gross_profit} net=€{net_profit}")


In [10]:
def main():
    """
    Main function to handle the input loop and commands.
    """
    data = load_data()
    print_help()
    while True:
        command = input("\nEnter a command: ").strip().lower()
        if command == "add":
            add_product(data)
            save_data(data)
        elif command == "list":
            list_products(data)
        elif command == "sale":
            register_sale(data)
            save_data(data)
        elif command == "profits":
            calculate_profits(data)
        elif command == "help":
            print_help()
        elif command == "exit":
            print("Bye bye")
            break
        else:
            print("Invalid command")
            print_help()

if __name__ == "__main__":
    main()

Available commands are:
add: add a product to the inventory
list: list all products in inventory
sale: register a completed sale
profits: show total profits
help: show available commands
exit: exit the program
Invalid value. Please enter a positive integer.
ADDED: 10 X help
Available commands are:
add: add a product to the inventory
list: list all products in inventory
sale: register a completed sale
profits: show total profits
help: show available commands
exit: exit the program
Bye bye
